<a href="https://colab.research.google.com/github/toanpt74/COLAB_RD/blob/main/AutoEncoder_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow import keras
import numpy as np
import random
import os
from matplotlib import pyplot as plt
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
print(tf.test.is_gpu_available())  # prints True
print(tf.__version__)  # prints '2.0.0-beta1'

batch_size = 32
latent_dim = 32
ROW = 28 #width
COL = 28 #height

input_shape=(ROW, COL, 1)
epochs=101
AUTO = tf.data.experimental.AUTOTUNE

def encoder(encoder_inputs):
    #Block 1
    x = layers.Conv2D(32, kernel_size=3, strides=1, padding='same')(encoder_inputs)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    # Block 2
    x = layers.Conv2D(64, kernel_size=3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    # Block 3
    x = layers.Conv2D(64, 3, 2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    # Block 4
    x = layers.Conv2D(64, 3, 1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    y = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    # Final Block
    flatten = layers.Flatten()(x)
    bottleneck = layers.Dense(16)(flatten)
    return bottleneck, y.shape


def decoder(x, conv_shape):
    units = conv_shape[1] * conv_shape[2] * conv_shape[3]

    x = layers.Dense(units)(x)
    x = layers.Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(x)

    # Block 1
    x = layers.Conv2DTranspose(64, 3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    # Block 2
    x = layers.Conv2DTranspose(64, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    # Block-3
    x = layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    # Block 4
    x = layers.Conv2DTranspose(32, 3, 1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    # Block 4
    decoder_outputs = layers.Conv2DTranspose(1, 3, 1, padding='same', activation='sigmoid', name='conv_transpose_4')(x)

    return decoder_outputs





inputs = keras.Input(shape=input_shape)
encoder_inputs, cov_shape = encoder(inputs)
decoder_outputs = decoder(encoder_inputs, cov_shape)
model = keras.Model(inputs,[decoder_outputs])
model.summary()

def ssim(input_img, output_img):
    return 1 - tf.reduce_mean(tf.image.ssim(input_img, tf.cast(output_img, tf.float64), max_val=1))

def loss(preds, real):
    #return 1 - tf.reduce_mean(tf.image.ssim(real, tf.cast(preds, tf.float64), max_val=1))
    return tf.reduce_mean(tf.square(tf.subtract(preds, real)))

optimizer = tf.keras.optimizers.Adam(lr=0.001)
objective = tf.keras.losses.SparseCategoricalCrossentropy()

(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255

num_samples = x_train.shape[0]
model_path =r'E:\ToanPT\1.Code_train_Unet\models'

def train_step(data):
    with tf.GradientTape() as tape:
        reconstruction = model(data, training=True)
        total_loss = loss(reconstruction, data)

    grads = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return total_loss

for epoch in range(epochs):
    for step in range(num_samples // batch_size):
        s = step * batch_size
        e = s + batch_size
        x_batch = x_train[s:e, :, :, tf.newaxis]
        total_loss = train_step(x_batch)
        print("-----------------")
        print(f"epoch: {epoch} step: {step}")
        print(f"total_loss: {total_loss}")
    if epoch % 5 == 0:
       model.save(f'E:\\ToanPT\\1.Code_train_Unet\\models\\autoencoder_{epoch}__{total_loss}', save_format="tf")


#model = tf.keras.models.load_model(r'E:\ToanPT\1.Code_train_Unet\models\autoencoder_100__0.004905893001705408')
